In [7]:
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline
['/Users/colinbull/Library/Application Support/JetBrains/Toolbox/apps/PyCharm-P/ch-0/241.15989.155/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Users/colinbull/Library/Application Support/JetBrains/Toolbox/apps/PyCharm-P/ch-0/241.15989.155/PyCharm.app/Contents/plugins/python/helpers/pydev', '/Users/colinbull/appdev/dfe/sfb/education-benchmarking-and-insights/data-pipeline', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/colinbull/Library/Caches/pypoetry/virtualenvs/fbit-data-pipeline-aJYNke-B-py3.12/lib/python3.12/site-packages']


# Comparator computation

This computes the Euclidean distance for each establishment from a base establishment of a consistent type (Academies, Maintained schools, SEN). Each establishment is given a weight against every other establishment and the top 60 for each establishment creates the comparator set for that establishment. This is repeated for all establishments in the establishment types set and then finally across all establishments across all establishment types.

In [8]:
import time 
import numpy as np
import pandas as pd
import src.pipeline.comparator_sets as comparators
import glob 
import os

start_time = time.time()
# Create and clean directory
from pathlib import Path
Path("output/comparator-sets").mkdir(parents=True, exist_ok=True)

# files = glob.glob("output/comparator-sets/*")
# for f in files:
#     os.remove(f)

# Prepare Academy and School Data

Here we prepare the academy and maintained school data by filling in missing values in NumberOfPupils, % Free School Meals and, % Sen with the mean (at this time). 

In [9]:
academy_data = comparators.prepare_data(pd.read_parquet("output/pre-processing/academies.parquet"))
ms_data = comparators.prepare_data(pd.read_parquet("output/pre-processing/maintained_schools.parquet"))
all_schools = comparators.prepare_data(pd.read_parquet("output/pre-processing/all_schools.parquet"))

FileNotFoundError: [Errno 2] No such file or directory: 'output/pre-processing/academies.parquet'

In [ ]:
def select_top_set(all_urns, all_regions, data, base_set_size=60, final_set_size=30):
    top_index = np.argsort(data, kind="stable")[:base_set_size]
    top_urns = all_urns[top_index]
    top_regions = all_regions[top_index]
    same_region = np.argwhere(top_regions == top_regions[0]).flatten()
    same_region_urns = top_urns[same_region]
    urns = np.append(same_region_urns, np.delete(top_urns, same_region)[:final_set_size - len(same_region_urns)])
    return all_urns, top_index, top_urns, same_region, same_region_urns

In [ ]:
data = ms_data[["OfstedRating (name)",
                 "Percentage SEN",
                 "Percentage Free school meals",
                 "Number of pupils",
                 "Total Internal Floor Area",
                 "Age Average Score",
                 "GOR (name)",
                 "SchoolPhaseType",
                 "Percentage Primary Need SPLD",
                 "Percentage Primary Need MLD",
                 "Percentage Primary Need PMLD",
                 "Percentage Primary Need SEMH",
                 "Percentage Primary Need SLCN",
                 "Percentage Primary Need HI",
                 "Percentage Primary Need MSI",
                 "Percentage Primary Need PD",
                 "Percentage Primary Need ASD",
                 "Percentage Primary Need OTH"]][ms_data["SchoolPhaseType"] == "Pupil referral unit"].copy()

group_data = data.reset_index().drop_duplicates().copy().groupby(['SchoolPhaseType']).agg(list)
data = comparators.compute_buildings_comparator(("Pupil referral unit", group_data.iloc[0]))[1]
all_urns = np.array(group_data["URN"].values[0])
all_regions = np.array(group_data["GOR (name)"].values[0])
select_top_set(all_urns, all_regions, data)

# Compute the pupil and building comparators

This creates the comparators sets across both academy and maintained schools

In [1]:
ms_comparators = comparators.compute_comparator_set(ms_data)
academy_comparators = comparators.compute_comparator_set(academy_data)
mixed_comparators = comparators.compute_comparator_set(all_schools)

NameError: name 'comparators' is not defined

In [1]:
ms_comparators.to_parquet("output/comparator-sets/ms_comparators.parquet")
# academy_comparators.to_parquet("output/comparator-sets/academy_comparators.parquet")
# academy_comparators.to_parquet("output/comparator-sets/mixed_comparators.parquet")
# ms_data.to_parquet("output/comparator-sets/maintained_schools.parquet")
# academy_data.to_parquet("output/comparator-sets/academies.parquet")
# all_schools.to_parquet("output/comparator-sets/all_schools.parquet")

NameError: name 'ms_comparators' is not defined

In [ ]:
ms_comparators.loc[100007]["Building"]

### Timing Keep at the bottom

In [ ]:
print(f'Processing Time: {time.time() - start_time} seconds')